In [91]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import config as cfg

In [78]:
df = pd.read_csv('../data/flights_samp.csv')

In [18]:
def split_numeric_categorical(df, numeric=True):
    '''    
    Return either numerical columns, or categorical columns in a data frame
    numeric: default True, return dataframe of all numerical columns    
    input: a data frame
    output: a data frame
    
    '''
    numeric_data = df.select_dtypes(include=[np.number])
    categorical_data = df.select_dtypes(include='category')
    if numeric:
        return numeric_data
    else:
        return categorical_data

In [82]:
def make_categorical(df, cols):
    '''
    Convert columns in `cols` to type `categorical`
    input: df - data frame, cols - a list of columns
    output: a copy of the data frame with the converted columns
    '''
    col_dict = {col:'category' for col in cols}
    df = df.astype(col_dict)
    return df

In [79]:
cols = ['mkt_carrier', 'tail_num']

In [80]:
df1 = make_categorical(df, cols)

In [83]:
def to_hhmmss(df_time_col):
    '''
    Change the format of the time in hhmm
    to hh:mm:ss, where ss is 00 in this case
    Input: `df_time_col`: a Pandas Series
    Return: a Pandas Series
    '''
    
    hhmm = []
    crs_hm = df_time_col.astype('str')
    for t in crs_hm:
        if len(t) == 1:
            hhmm.append('0' + t + ':00:00')
        elif (len(t) == 2) & (t < '24'):
            hhmm.append(t + ':00:00')
        elif (len(t) == 2) & (t > '24'):
            hhmm.append('00:' + t + ':00')
        elif len(t) == 3:
            hhmm.append('0' + t[0] + ':' + t[1:] + ':00')
        else:
            hhmm.append(t[:2] + ':' + t[2:] + ":00")
    return hhmm

In [84]:
df['crs_dep_time_hhmm'] = to_hhmmss(df.crs_dep_time)

In [85]:
def to_city_state(city_state_col):
    '''
    Change the format of `city1/city2, state`
    to `city1, state` 
    Input: A Pandas Series
    Output: A Pandas Series
    '''
    return pd.Series(map(lambda x: x[0].split('/')[0] + ',' + x[-1].strip() ,
                    city_state_col.str.split(',')))

In [86]:
df.origin_city_name = to_city_state(df.origin_city_name)

In [90]:
def request_weather(df):
    '''
    request weather information
    input: a data frame in which
        - first column: time 
        - second column: location
        - third column: date
    output: a dictionary 
    '''
    unit_group="us"
    api_key = cfg.visual_crossing['api_key']
    requestUrl = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline'
    weather_dict = { 'conditions': [] }
    count = 1
    for row in df.values:
        print('Query Count: ', count)
        time = row[0]
        loc = row[1]
        date = row[2]
        url = requestUrl + '/' + loc + '/' + date + 'T' + time + '?key=' + api_key   
        res = requests.get(url)
        if res.status_code == 200:
            weather_json = res.json()
            try:
                condition = weather_json['days'][0]['conditions']
            except:
                condition = 'NA'
            else:        
                weather_dict['conditions'].append(condition)
        else:
            weather_dict['conditions'].append('NA')
            print('Error: ', res.status_code)    
        count += 1
    return weather_dict